<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Mini_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. Nettoyage et installation des dépendances
!pip uninstall -y langchain langchain-community
!pip install --upgrade pip
!pip install langchain==0.1.0 langchain-community==0.0.10
!pip install streamlit llama-cpp-python huggingface_hub pyngrok streamlit-chat


Found existing installation: langchain 0.1.0
Uninstalling langchain-0.1.0:
  Successfully uninstalled langchain-0.1.0
Found existing installation: langchain-community 0.0.10
Uninstalling langchain-community-0.0.10:
  Successfully uninstalled langchain-community-0.0.10
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
Using cached langchain_community-0.0.10-py3-none-any.whl (1.5 MB)


In [21]:
# Imports nécessaires
import streamlit as st
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from huggingface_hub import hf_hub_download
import os
import json
from datetime import datetime

# Fonction de gestion de la mémoire permanente
class CompanyMemory:
    def __init__(self):
        self.memory_file = 'company_memory.json'
        self.memory = self.load_memory()

    def load_memory(self):
        try:
            with open(self.memory_file, 'r') as f:
                return json.load(f)
        except:
            return {
                "clients": [
                    "Gucci", "Cartier", "Loewe", "L'Oréal", "Porsche", "Peugeot",
                    "Cécile Bahnsen", "Teenage Engineering", "Spitfire Audio",
                    "Dior", "Ventes Privées", "Aldebaran"
                ],
                "projects": [],
                "updates": []
            }

    def save_memory(self):
        with open(self.memory_file, 'w') as f:
            json.dump(self.memory, f, indent=2)

    def add_client(self, client):
        if client not in self.memory["clients"]:
            self.memory["clients"].append(client)
            self.memory["updates"].append({
                "date": datetime.now().isoformat(),
                "type": "new_client",
                "content": f"Nouveau client ajouté : {client}"
            })
            self.save_memory()

    def add_project(self, project_info):
        self.memory["projects"].append({
            "date": datetime.now().isoformat(),
            "content": project_info
        })
        self.memory["updates"].append({
            "date": datetime.now().isoformat(),
            "type": "new_project",
            "content": project_info
        })
        self.save_memory()

    def get_clients(self):
        return sorted(self.memory["clients"])

    def get_recent_updates(self, limit=5):
        return sorted(self.memory["updates"],
                     key=lambda x: x["date"],
                     reverse=True)[:limit]

# Initialisation de la mémoire de l'entreprise
if 'company_memory' not in st.session_state:
    st.session_state.company_memory = CompanyMemory()

# Configuration du LLM
@st.cache_resource
def init_llm():
    model_path = hf_hub_download(
        repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
        filename="mistral-7b-instruct-v0.1.Q4_0.gguf",
        resume_download=True
    )
    callbacks = CallbackManager([StreamingStdOutCallbackHandler()])
    return LlamaCpp(
        model_path=model_path,
        temperature=0.7,
        max_tokens=4096,
        n_ctx=4096,
        n_batch=512,
        callback_manager=callbacks,
        verbose=True,
        streaming=True
    )

# Configuration de l'interface
st.set_page_config(page_title="Solweig & Izar AI Assistant", page_icon="🎨", layout="wide")
st.title("🎨 Assistant IA Solweig & Izar")

# Système de prompt dynamique
def get_system_prompt():
    memory = st.session_state.company_memory
    clients = memory.get_clients()
    recent_updates = memory.get_recent_updates()

    return f"""Tu es l'assistant de Solweig & Izar. Instructions importantes :

1. Réponds de manière directe et concise
2. Ne génère jamais de texte commençant par "Human:", "AI:", ou "Assistant:"
3. N'ajoute pas de formules de politesse superflues
4. Ne mentionne pas tes capacités de mémorisation
5. Pour les listes de clients, présente-les par catégorie

Clients actuels : {', '.join(clients)}

Mises à jour récentes :
{chr(10).join([f"- {update['content']}" for update in recent_updates])}"""

# Modification du traitement des messages pour éviter les réponses automatiques
if prompt := st.chat_input(""):
    # Nettoyage du prompt pour éviter les préfixes non désirés
    clean_prompt = prompt.replace("Human:", "").replace("AI:", "").strip()

    st.session_state.messages.append({"role": "user", "content": clean_prompt})

    try:
        with st.chat_message("assistant"):
            response = chain.run(input=clean_prompt)
            # Nettoyage de la réponse
            clean_response = response.replace("Human:", "").replace("AI:", "").replace("Assistant:", "").strip()
            # Suppression des phrases de politesse superflues
            clean_response = re.sub(r"Si vous avez d'autres informations.*$", "", clean_response)
            clean_response = re.sub(r"N'hésitez pas à.*$", "", clean_response)
            st.markdown(clean_response)
            st.session_state.messages.append({"role": "assistant", "content": clean_response})
    except ValueError as e:
        if "exceed context window" in str(e):
            response = chain.run(input=clean_prompt[:1000])
            clean_response = response.replace("Human:", "").replace("AI:", "").replace("Assistant:", "").strip()
            clean_response = re.sub(r"Si vous avez d'autres informations.*$", "", clean_response)
            st.markdown(clean_response)
            st.session_state.messages.append({"role": "assistant", "content": clean_response})



2025-04-04 09:07:38.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 09:07:38.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [22]:
# Nettoyage des processus
from pyngrok import ngrok, conf
ngrok.kill()

import time
time.sleep(2)

# Configuration de ngrok
!ngrok authtoken 2vAdrpYdJA16CJ9rJ5PC653n6b0_goQTqTbrL9TYLjcPPKgN

# Lancement de l'application
try:
    # Lancement de Streamlit
    !streamlit run app.py &>/dev/null &
    time.sleep(5)  # Attente du démarrage de Streamlit

    # Création d'un nouveau tunnel
    public_url = ngrok.connect(addr="8501", proto="http")
    print(f"🚀 Votre application est accessible sur : {public_url}")
except Exception as e:
    print(f"⚠️ Erreur : {str(e)}")



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Votre application est accessible sur : NgrokTunnel: "https://b081-34-60-232-134.ngrok-free.app" -> "http://localhost:8501"
